In [48]:
#import requiered libraries
import os
import warnings
import sys

import dvc.api
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet, LogisticRegression
from sklearn.preprocessing import LabelEncoder
from fast_ml.model_development import train_valid_test_split
from sklearn.tree import DecisionTreeClassifier
from urllib.parse import urlparse
from sklearn import preprocessing


In [49]:
import warnings
warnings.filterwarnings('ignore')

In [51]:
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from train_classifiers import TrainingClassifier 


In [26]:
cleaned_df  = pd.read_csv('../data/clean_data.csv')

In [6]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1243 entries, 0 to 1242
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   1243 non-null   int64 
 1   auction_id   1243 non-null   object
 2   experiment   1243 non-null   object
 3   date         1243 non-null   object
 4   hour         1243 non-null   int64 
 5   device_make  1243 non-null   object
 6   platform_os  1243 non-null   int64 
 7   browser      1243 non-null   object
 8   aware        1243 non-null   int64 
dtypes: int64(4), object(5)
memory usage: 87.5+ KB


In [52]:
# Load data from dvc using the dvc.api.Dataset class
path = 'data/AdSmartABdata.csv'
repo = 'https://github.com/Group-12-A-B-Hypothesis-Testing/abtest-mlops'
version = 'c1dfb93'
re_url = dvc.api.get_url(path=path, repo=repo, rev=version)
print(re_url)

gdrive://1Qwhs22KV5-hdeMZ-1coPd65tJ7SQ3IJd/9c/00ecff555480cd97b707617d639d3f


In [ ]:
data_os = pd.read_csv(re_url, sep=',')

Label Encoding the Data

In [35]:
def encode_labels(df):
    date_encoder = preprocessing.LabelEncoder()
    device_encoder = preprocessing.LabelEncoder()
    browser_encoder = preprocessing.LabelEncoder()
    experiment_encoder = preprocessing.LabelEncoder()
    aware_encoder = preprocessing.LabelEncoder()
    
    df['date'] = date_encoder.fit_transform(df['date'])
    df['device_make'] = device_encoder.fit_transform(df['device_make'])
    df['browser'] = browser_encoder.fit_transform(df['browser'])
    df['experiment'] = experiment_encoder.fit_transform(df['experiment'])
    df['browser'] = aware_encoder.fit_transform(df['browser'])
    df['aware'] = aware_encoder.fit_transform(df['aware'])
    
    return df    

Spliting the data by browser and platform_os column

In [20]:
def feature_data(df):
    
    browser_feature_df = df[["experiment", "hour", "date", 'device_make', 'browser', 'aware']] 
    platform_feature_df = df[["experiment", "hour", "date", 'device_make', 'platform_os', 'aware']] 

    return browser_feature_df, platform_feature_df

In [24]:
def save_encoded_df(df):
    
    broweser_df, platfrom_df = feature_data(df)
    broweser_df.save_csv("../Data/browser_featured_data.csv")
    platfrom_df.save_csv(platfrom_df, "../data/platform_featured_data.csv")

In [36]:
cleaned_data = encode_labels(cleaned_df)
cleaned_browser_df, cleaned_platform_df = feature_data(cleaned_data)

In [37]:
cleaned_browser_df.head()

,experiment,hour,date,device_make,browser,aware
0,1,2,3,4,2,0
1,1,16,2,13,1,1
2,1,8,4,13,1,0
3,0,4,6,43,4,1
4,0,15,1,13,1,0


In [38]:
Y_col = 'aware'
X_cols = cleaned_browser_df.loc[:, cleaned_browser_df.columns != Y_col].columns

X_train,X_test,y_train,y_test=train_test_split(cleaned_browser_df[X_cols], cleaned_browser_df[Y_col],\
                                                test_size=0.1, random_state=42)

### Training Browser Featured Data

#### Training with Logistic Regression